# 1) BST
https://leetcode.com/problems/search-in-a-binary-search-tree/description/


In [ ]:
from typing import Optional

# Definition for a binary tree node
class TreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

class Solution:
    def searchBST(self, root: list, val: int) -> Optional[TreeNode]:
        if root is None:
            # print(f"Return 1: {root}")
            return
        elif root.val == val:
            # print(f"FOUND!!! root.val ({root.val}) == val ({val})")
            return root
        elif val > root.val: 
            # print(f"val ({val}) > root.val ({root.val}) --> RIGHT")
            return self.searchBST(root.right, val = val)
        elif val < root.val:
            # print(f"val ({val}) < root.val ({root.val}) --> LEFT")
            return self.searchBST(root.left, val = val)
        else:
            # print(f"Unknown Return")
            return

In [3]:
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'matplotlib'